In [4]:
!unzip test.zip -d my_data

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: my_data/test/REAL/0500 (10).jpg  
  inflating: my_data/test/REAL/0500 (2).jpg  
  inflating: my_data/test/REAL/0500 (3).jpg  
  inflating: my_data/test/REAL/0500 (4).jpg  
  inflating: my_data/test/REAL/0500 (5).jpg  
  inflating: my_data/test/REAL/0500 (6).jpg  
  inflating: my_data/test/REAL/0500 (7).jpg  
  inflating: my_data/test/REAL/0500 (8).jpg  
  inflating: my_data/test/REAL/0500 (9).jpg  
  inflating: my_data/test/REAL/0500.jpg  
  inflating: my_data/test/REAL/0501 (10).jpg  
  inflating: my_data/test/REAL/0501 (2).jpg  
  inflating: my_data/test/REAL/0501 (3).jpg  
  inflating: my_data/test/REAL/0501 (4).jpg  
  inflating: my_data/test/REAL/0501 (5).jpg  
  inflating: my_data/test/REAL/0501 (6).jpg  
  inflating: my_data/test/REAL/0501 (7).jpg  
  inflating: my_data/test/REAL/0501 (8).jpg  
  inflating: my_data/test/REAL/0501 (9).jpg  
  inflating: my_data/test/REAL/0501.jpg  
  inflating: my_da

In [1]:
!unzip train.zip -d my_data

Archive:  train.zip
   creating: my_data/train/
   creating: my_data/train/FAKE/
  inflating: my_data/train/FAKE/0bAvf1FcUP.jpeg  
  inflating: my_data/train/FAKE/0xvbLyG0L2.png  
  inflating: my_data/train/FAKE/1000 (10).jpg  
  inflating: my_data/train/FAKE/1000 (2).jpg  
  inflating: my_data/train/FAKE/1000 (3).jpg  
  inflating: my_data/train/FAKE/1000 (4).jpg  
  inflating: my_data/train/FAKE/1000 (5).jpg  
  inflating: my_data/train/FAKE/1000 (6).jpg  
  inflating: my_data/train/FAKE/1000 (7).jpg  
  inflating: my_data/train/FAKE/1000 (8).jpg  
  inflating: my_data/train/FAKE/1000 (9).jpg  
  inflating: my_data/train/FAKE/1000.jpg  
  inflating: my_data/train/FAKE/1001 (10).jpg  
  inflating: my_data/train/FAKE/1001 (2).jpg  
  inflating: my_data/train/FAKE/1001 (3).jpg  
  inflating: my_data/train/FAKE/1001 (4).jpg  
  inflating: my_data/train/FAKE/1001 (5).jpg  
  inflating: my_data/train/FAKE/1001 (6).jpg  
  inflating: my_data/train/FAKE/1001 (7).jpg  
  inflating: my_data/tr

# Импорт библиотек

In [2]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.applications.efficientnet import preprocess_input
from tensorflow import keras
from keras.models import load_model

In [3]:
import sys
import PIL
print(f'python== {sys.version}')
print(f'pandas== {pd.__version__}')
print(f'numpy== {np.__version__}')
print(f'tensorflow== {tf.__version__}')
print(f'pillow== {PIL.__version__}')

python== 3.10.12 (main, Jul 29 2024, 16:56:48) [GCC 11.4.0]
pandas== 2.1.4
numpy== 1.26.4
tensorflow== 2.17.0
pillow== 9.4.0


# Константы

In [5]:
IMG_SIZE = 224
DATA_FOLDER = 'C://Users//Lilya//Downloads//fake-or-real//data//'
TRAIN_PATH = os.path.join(DATA_FOLDER, 'train//')
TEST_PATH = os.path.join(DATA_FOLDER, 'test//')
IMG_PATH = os.path.join(DATA_FOLDER, 'my_images//')
BATCH_SIZE = 16
VALIDATION_SPLIT = 0.2

# Импорт данных

In [36]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  TRAIN_PATH,
  validation_split=VALIDATION_SPLIT,
  subset="training",
  seed=123,
  image_size=(IMG_SIZE, IMG_SIZE),
  batch_size=BATCH_SIZE)

val_ds = tf.keras.utils.image_dataset_from_directory(
  TRAIN_PATH,
  validation_split=VALIDATION_SPLIT,
  subset="validation",
  seed=123,
  image_size=(IMG_SIZE, IMG_SIZE),
  batch_size=BATCH_SIZE)

Found 1412 files belonging to 2 classes.
Using 1130 files for training.
Found 1412 files belonging to 2 classes.
Using 282 files for validation.


# EfficientNet

## Обучение модели

In [42]:
# Building the Transfer Learning model using EfficientNetV2B0
EfficientNet_base_model = tf.keras.applications.EfficientNetV2B0(
    include_top = False,
    weights = 'imagenet',
    input_shape = (IMG_SIZE, IMG_SIZE, 3),
    pooling = 'max'
)
EfficientNet_base_model.trainable = True

# Create a new model on top of the EfficientNet base
inputs = tf.keras.Input(shape = (IMG_SIZE, IMG_SIZE, 3))
x = EfficientNet_base_model(inputs, training = False)
x = BatchNormalization(axis = -1, momentum = 0.99, epsilon = 0.001)(x)
x = Dropout(rate = .4, seed = 512)(x)
x = Dense(64, activation = 'relu')(x)
outputs = Dense(1, activation = 'sigmoid')(x)
EfficientNet_model = tf.keras.Model(inputs, outputs)

# Compile the Transfer Learning model
EfficientNet_model.compile(
    optimizer = tf.keras.optimizers.Adamax(learning_rate = .001),
    loss = tf.keras.losses.BinaryCrossentropy()
)

# Build the Transfer Learning model so we can see a summary
EfficientNet_model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_15 (InputLayer)          │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ efficientnetv2-b0 (Functional)       │ (None, 1280)                │       5,919,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 64)                  │          81,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,001,361 (22.89 MB)

 Trainable params: 5,940,753 (22.66 MB)

 Non-trainable params: 60,608 (236.75 KB)

In [6]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',
                                                  patience = 25,
                                                  restore_best_weights = True,
                                                  min_delta=0.01)

In [43]:
# with augmantation & normalization
EfficientNet_model_history = EfficientNet_model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 100,
    verbose = 1,
    callbacks = [early_stopping]
)

Epoch 1/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 149s 910ms/step - loss: 1.3203 - val_loss: 0.2775
Epoch 2/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 41s 108ms/step - loss: 0.3212 - val_loss: 0.1789
Epoch 3/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 8s 112ms/step - loss: 0.2306 - val_loss: 0.1487
Epoch 4/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 11s 128ms/step - loss: 0.1705 - val_loss: 0.1517
Epoch 5/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 7s 91ms/step - loss: 0.1182 - val_loss: 0.1251
Epoch 6/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 11s 101ms/step - loss: 0.1130 - val_loss: 0.1463
Epoch 7/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 11s 113ms/step - loss: 0.0756 - val_loss: 0.1358
Epoch 8/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 10s 113ms/step - loss: 0.0921 - val_loss: 0.1321
Epoch 9/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 9s 94ms/step - loss: 0.0658 - val_loss: 0.1306
Epoch 10/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 8s 115ms/step - loss: 0.0537 - val_loss: 0.1583
Epoch 11/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 7s 94ms/step - loss: 0.0421 - val_loss: 0.1739
Epoch 12/100
71/71 ━━━━━━━━━━━━━━━━━━

## Оценка модели

In [6]:
pred_ds = tf.keras.utils.image_dataset_from_directory(
  TEST_PATH,
  # labels=None,
  seed = 512,
  image_size = (IMG_SIZE, IMG_SIZE),
  batch_size = BATCH_SIZE, shuffle=False)

Found 20000 files belonging to 2 classes.


In [8]:
EfficientNet_model.evaluate(pred_ds)

1250/1250 ━━━━━━━━━━━━━━━━━━━━ 35s 20ms/step - loss: 0.2617


0.6115557551383972

In [40]:
EfficientNet_model.evaluate(val_ds)

18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0761


0.07517015188932419

## Сохранение модели

### В формате tf

In [41]:
EfficientNet_model.save('EfficientNet', save_format='tf')

In [ ]:
!zip -r /content/EfficientNet.zip /content/EfficientNet

  adding: content/EfficientNet/ (stored 0%)
  adding: content/EfficientNet/fingerprint.pb (stored 0%)
  adding: content/EfficientNet/variables/ (stored 0%)
  adding: content/EfficientNet/variables/variables.data-00000-of-00001 (deflated 6%)
  adding: content/EfficientNet/variables/variables.index (deflated 77%)
  adding: content/EfficientNet/saved_model.pb (deflated 91%)
  adding: content/EfficientNet/keras_metadata.pb (deflated 96%)
  adding: content/EfficientNet/assets/ (stored 0%)


### В формате h5

In [ ]:
EfficientNet_model.save('EfficientNet.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Предсказание

In [7]:
EfficientNet_model = load_model('EfficientNet.h5')

### Для предсказания тестового набора данных

In [ ]:
test = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv'))
test['new_id'] = test.id.str.split('.').str.get(0)
image_list = os.listdir(TEST_PATH)
data_image = pd.DataFrame(image_list, columns = ['name_ext'])
data_image['name'] = data_image.name_ext.str.split('.').str.get(0)
test = test.merge(data_image, how='left', left_on = 'new_id', right_on='name')

In [ ]:
pred_ds = tf.keras.utils.image_dataset_from_directory(
  TEST_PATH,
  labels=None,
  seed = 512,
  image_size = (IMG_SIZE, IMG_SIZE),
  batch_size = BATCH_SIZE, shuffle=False)

In [ ]:
filenames = pred_ds.file_paths
file_names = []

# Iterate through all image files in the specified directory
for file in filenames:
    label = str(file).split('/')[-1]  # Extract the label from the file path
    file_names.append(label)

In [ ]:
predictions = EfficientNet_model.predict(pred_ds)
predictions = predictions[:, 0]
pred_df = pd.DataFrame(list(zip(file_names, predictions)), columns = ['Name', 'target'])

32/32 [==============================] - 18s 451ms/step


In [ ]:
res = test.merge(pred_df, left_on='name_ext', right_on = 'Name', how='left')
res = res[['id', 'target']]
res = res[res.target.isna()==False]
res.target = res.target.apply(lambda x: 1-x)

In [ ]:
res.to_csv('predictions.csv', index=False)

### Для предсказания одной картинки

In [ ]:
# Загрузка изображения
img = load_img(os.path.join(IMG_PATH, '0a1KuiILS4.jpg'),
               target_size=(IMG_SIZE, IMG_SIZE))  # Указать путь к изображению и размер

# Преобразование изображения в массив numpy
img_array = img_to_array(img)
# Расширение размерности на 1, чтобы модель могла обработать
img_array = np.expand_dims(img_array, axis=0)
# Предварительная обработка массива
processed_img = preprocess_input(img_array)

predictions = EfficientNet_model.predict(processed_img)
if predictions < 0.5:
  print('Я думаю, что эта картинка сгенерирована нейросетью')
else:
  print('Я думаю, что эта картинка сделана человеком')

1/1 [==============================] - 6s 6s/step
Я думаю, что эта картинка сделана человеком
